Name: Dwi Fitria Astari Lubis

CUSTOMER CHURN PREDICTION

Develop a model to predict customer churn for a subscription based service or business. Use historical customer data, including features like usage behavior and customer demographics, and try algorithms like Logistic Regression, Random Forests, or Gradient Boosting to predict churn.

source of dataset: https://www.kaggle.com/datasets/shantanudhakadd/bank-customer-churn-prediction
It is the dataset of a U.S. bank customer for getting the information that , this particular customer will leave bank or not.

In [33]:
#Library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score
from sklearn import svm
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import joblib

In [3]:
#Load dataset and Profiling data
df=pd.read_csv('C:\\Users\\Dwi Fitria\\Intern\\Encryptix\\Churn_Modelling.csv')

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [11]:
df=df.drop(['RowNumber', 'CustomerId', 'Surname'],axis=1)

In [12]:
df.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [13]:
#Encoding Label
categorical_cols = ['Geography', 'Gender', 'Balance', 'EstimatedSalary']

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [14]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0,1,1,1,5068,1
1,608,2,0,41,1,743,1,0,1,5639,0
2,502,0,0,42,8,5793,3,1,0,5707,1
3,699,0,0,39,1,0,2,0,0,4704,0
4,850,2,0,43,2,3696,1,1,1,3925,0


In [17]:
df.describe()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,0.746300,0.545700,38.921800,5.012800,2036.788100,1.530200,0.70550,0.515100,4998.621200,0.203700
std,96.653299,0.827529,0.497932,10.487806,2.892174,2125.232536,0.581654,0.45584,0.499797,2886.711202,0.402769
min,350.000000,0.000000,0.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000
25%,584.000000,0.000000,0.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,2498.750000,0.000000
50%,652.000000,0.000000,1.000000,37.000000,5.000000,1383.500000,1.000000,1.00000,1.000000,4998.500000,0.000000
75%,718.000000,1.000000,1.000000,44.000000,7.000000,3882.250000,2.000000,1.00000,1.000000,7498.250000,0.000000
max,850.000000,2.000000,1.000000,92.000000,10.000000,6381.000000,4.000000,1.00000,1.000000,9998.000000,1.000000


In [18]:
df['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

7963 person will not leave the bank
2037 person will leave bank

In [19]:
#Handling imbalanced data with SMOTE
X=df.drop(['Exited'],axis=1)
y=df['Exited']
X_res,y_res=SMOTE().fit_resample(X,y)
y_res.value_counts()

Exited
1    7963
0    7963
Name: count, dtype: int64

In [20]:
#train data
X_train,X_test,y_train,y_test=train_test_split(X_res,y_res,test_size=0.20,random_state=42)
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)
X_train

array([[-1.45972397,  0.43267073, -0.84374487, ..., -1.2773753 ,
        -0.76948163,  1.71716086],
       [ 1.08360015, -0.88699054,  1.18519239, ..., -1.2773753 ,
         1.29957619,  0.75932586],
       [-0.42074901,  0.43267073,  1.18519239, ..., -1.2773753 ,
        -0.76948163, -1.59314751],
       ...,
       [ 0.19614237, -0.88699054,  1.18519239, ...,  0.78285528,
        -0.76948163, -0.05163182],
       [-0.17182793,  0.43267073, -0.84374487, ...,  0.78285528,
        -0.76948163,  1.46273594],
       [ 0.52082205,  0.43267073,  1.18519239, ...,  0.78285528,
         1.29957619, -0.83335326]])

In [24]:
#LogisticRegression
log = LogisticRegression()
log.fit(X_train,y_train)

#Make Prediction
y_pred = log.predict(X_test)

In [25]:
#Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:",
 f1)

Accuracy: 0.7661644695543001
Precision: 0.7525
Recall: 0.7752736638763683
F1-score: 0.7637170948303204


In [34]:
#SupportVectorClassifier
svm = svm.SVC()
svm.fit(X_train,y_train)
y_pred1 = svm.predict(X_test)
accuracy_score(y_test,y_pred1)

0.8408662900188324

In [36]:
precision_score(y_test,y_pred1)

np.float64(0.8369845360824743)

In [52]:
#RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred2=rf.predict(X_test)
accuracy_score(y_test,y_pred2)
X_res=sc.fit_transform(X_res)
rf.fit(X_res,y_res)


RandomForestClassifier()

In [53]:
precision_score(y_test,y_pred2)

np.float64(0.8462507876496534)

In [54]:
#GradienBoosingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(X_train,y_train)
y_pred3=gbc.predict(X_test)
accuracy_score(y_test,y_pred3)

0.8364720652856246

In [55]:
precision_score(y_test,y_pred3)

np.float64(0.8341968911917098)

In [56]:
final_data_accuracy_score=pd.DataFrame({'Models':['LR','SVC','RF','GBC'],
                         'ACC':[accuracy_score(y_test,y_pred),
                                accuracy_score(y_test,y_pred1),
                                accuracy_score(y_test,y_pred2),
                                accuracy_score(y_test,y_pred3)]})
final_data_accuracy_score

,Models,ACC
0,LR,0.766164
1,SVC,0.840866
2,RF,0.857502
3,GBC,0.836472


In [57]:
final_data_precision_score=pd.DataFrame({'Models':['LR','SVC','RF','GBC'],
                         'pre':[precision_score(y_test,y_pred),
                                precision_score(y_test,y_pred1),
                                precision_score(y_test,y_pred2),
                                precision_score(y_test,y_pred3)]})
final_data_precision_score

,Models,pre
0,LR,0.752500
1,SVC,0.836985
2,RF,0.846251
3,GBC,0.834197


In [59]:
joblib.dump(rf,'Churn_predict_model')
model=joblib.load('Churn_predict_model')
df.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')

In [62]:
if(model.predict([[850,2,0,43,2,3696,1,1,1,3925]])==1):
    print('Customer can leave the bank')
else:
    print('Customer will not leave the bank')

Customer will not leave the bank


approved: sample data number 4, exited=0 (customer will not leave the bank)